In [37]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Import Counter module from collections 
from collections import Counter

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [38]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
csv_path = Path("./Resources/lending_data.csv")
df_lending = pd.read_csv(csv_path)

# Review the DataFrame
print(df_lending.shape)
df_lending.tail()

(77536, 8)


,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
77531,19100.0,11.261,86600,0.653580,12,2,56600,1
77532,17700.0,10.662,80900,0.629172,11,2,50900,1
77533,17600.0,10.595,80300,0.626401,11,2,50300,1
77534,16300.0,10.068,75300,0.601594,10,2,45300,1
77535,15600.0,9.742,72300,0.585062,9,2,42300,1


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [39]:
# Separate the data into labels and features
# Separate the y variable, the labels
y = df_lending["loan_status"]

# Separate the X variable, the features
X = df_lending.drop(columns = "loan_status")

In [40]:
# Review the y variable Series
y

0        0
1        0
2        0
3        0
4        0
        ..
77531    1
77532    1
77533    1
77534    1
77535    1
Name: loan_status, Length: 77536, dtype: int64

In [41]:
# Review the X variable DataFrame
print(X.shape)
X.tail()

(77536, 7)


,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
77531,19100.0,11.261,86600,0.653580,12,2,56600
77532,17700.0,10.662,80900,0.629172,11,2,50900
77533,17600.0,10.595,80300,0.626401,11,2,50300
77534,16300.0,10.068,75300,0.601594,10,2,45300
77535,15600.0,9.742,72300,0.585062,9,2,42300


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [42]:
# Check the balance of our target values
print(Counter(y))
y.value_counts()

Counter({0: 75036, 1: 2500})


loan_status
0    75036
1     2500
Name: count, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [43]:
# Split the data using train_test_split
# Assign a random_state of 1 to the function
# Also using parameter stratify to handle imbalance
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(58152, 7)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [44]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver = 'lbfgs', max_iter = 100, random_state = 1)

# Fit the model using training data
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [45]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
results = pd.DataFrame({"Predicted": predictions, "Actual": y_test}).reset_index(drop=True)
results.head(10)

,Predicted,Actual
0,0,0
1,0,1
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [46]:
# Print the balanced_accuracy score of the model
print(balanced_accuracy_score(y_test, predictions))

0.9442676901753825


In [47]:
# Generate a confusion matrix for the model
print(confusion_matrix(y_test, predictions))

[[18679    80]
 [   67   558]]


In [48]:
# Print the classification report for the model
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      0.89      0.88       625

    accuracy                           0.99     19384
   macro avg       0.94      0.94      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** 
It has a better performance predicting healthy loans (~1.00) than high-risk loans (~0.87), as can be seen in the confusion matrix and in the classification report. 

It should be noted the imbalance between labels, which can result in prediction imbalance. 
Label for healthy loan is around 30 times higher than label for high-risk loan: 
> _Counter({0: 75036, 1: 2500})_

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [49]:
# Instantiate the random oversampler model
# Assign a random_state parameter of 1 to the model
sampler = RandomOverSampler(random_state=1)
X_res, y_res = sampler.fit_resample(X, y)

# Fit the original training data to the random_oversampler model
sampler.fit(X_train, y_train)

RandomOverSampler(random_state=1)

In [50]:
# Count the distinct values of the resampled labels data
print(Counter(y_res))
y_res.value_counts()

Counter({0: 75036, 1: 75036})


loan_status
0    75036
1    75036
Name: count, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [51]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
X_res_train, X_res_test, y_res_train, y_res_test = train_test_split(X_res, y_res, random_state=1)
classifier_res = LogisticRegression(solver = 'lbfgs', max_iter = 100, random_state = 1)

# Fit the model using the resampled training data
print(classifier_res.fit(X_res_train, y_res_train))

# Make a prediction using the testing data
predictions_res = classifier_res.predict(X_res_test)
pd.DataFrame({"Prediction (res)": predictions_res, "Actual (res)": y_res_test})

LogisticRegression(random_state=1)


,Prediction (res),Actual (res)
31735,0,0
85950,1,1
114137,1,1
32165,0,0
26504,0,0
...,...,...
70523,0,0
15683,0,0
144522,1,1
55222,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [52]:
# Print the balanced_accuracy score of the model 
print(balanced_accuracy_score(y_res_test, predictions_res))

0.9950160462226244


In [53]:
# Generate a confusion matrix for the model
print(confusion_matrix(y_res_test, predictions_res))

[[18810    95]
 [   92 18521]]


In [54]:
# Print the classification report for the model
print(classification_report(y_res_test, predictions_res))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18905
           1       0.99      1.00      0.99     18613

    accuracy                           1.00     37518
   macro avg       1.00      1.00      1.00     37518
weighted avg       1.00      1.00      1.00     37518



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** It has a comparable performance predicting both healthy loans (~1.00) and high-risk loans (~0.99), as can be seen in the confusion matrix and in the classification report. 

In summary, the Logistic Regression model has better performance when fit using balanced labels. 